<a href="https://colab.research.google.com/github/41371125h-chinrouzhen/114-1-PL/blob/main/HW5_%E9%A4%90%E5%BB%B3%E6%B1%BA%E7%AD%96%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
!pip -q install gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 \
               gradio pandas beautifulsoup4 google-generativeai python-dateutil \
               google-search-results

In [66]:
import os
import pandas as pd
import numpy as np
import random
import gradio as gr
import asyncio
import json
import google.generativeai as genai
from google.colab import userdata, auth
from google.auth import default
from serpapi import GoogleSearch

try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=GEMINI_API_KEY)

    SYSTEM_PROMPT = (
        "您是一位幽默且極具效率的午餐決策顧問。您的任務是根據提供的餐廳清單，"
        "分析其優缺點（如價格、評分、特色）並總結，幫助使用者在 3 家中快速做出決定。"
        "請以輕鬆口吻，用 **繁體中文** 撰寫一個約 **100 字** 的決策建議摘要，不超過 150 字。"
        "請避免使用 Markdown 標題和編號清單。"
    )

    model = genai.GenerativeModel(
        'models/gemini-2.5-flash',
        system_instruction=SYSTEM_PROMPT
    )

    print("Gemini API 設定成功！")
except Exception as e:
    print(f"Gemini API 設定失敗：{e}。AI 摘要功能將無法使用。")
    model = None

try:
    SERPAPI_KEY = userdata.get('SERPAPI_KEY')
    if SERPAPI_KEY is None:
        raise ValueError("未能在 Colab Secrets 中找到 'SERPAPI_KEY'")
    print("SerpApi API 設定成功！")
except Exception as e:
    print(f"SerpApi API 設定失敗：{e}。你將無法搜尋餐廳。")
    SERPAPI_KEY = None

Gemini API 設定成功！
SerpApi API 設定成功！


In [67]:
def parse_serpapi_to_df(serpapi_results):
    """
    輔助函式 1：(核心) 將 SerpApi 回傳的 JSON 轉換為 Pandas DataFrame
    """
    local_results = serpapi_results.get("local_results", [])

    restaurants = []
    for place in local_results:

        price_str = place.get('price', '$')

        open_state = place.get('open_state', '').lower()
        hours_str = place.get('hours', '').lower()
        open_now = "是" if (open_state == 'open' or 'open ' in hours_str or '營業中' in hours_str) else "否"

        types = place.get('types', [])
        excluded_types = {'restaurant', 'food', 'point_of_interest', 'establishment'}
        features = [t for t in types if t not in excluded_types]
        feature_str = ', '.join(features[:3])

        lat = place.get('gps_coordinates', {}).get('latitude')
        lng = place.get('gps_coordinates', {}).get('longitude')

        restaurants.append({
            '餐廳名稱': place.get('title'),
            '價位': price_str,
            '評分': place.get('rating', 'N/A'),
            '總評論數': place.get('reviews', 'N/A'),
            '是否營業': open_now,
            '特色描述': feature_str,
            'place_id': place.get('data_cid'),
            'latitude': lat,
            'longitude': lng,
            'address': place.get('address') # !! 新增：抓取地址
        })

    return pd.DataFrame(restaurants)

In [68]:
async def generate_summary(ai_input):
    """
    呼叫 LLM 產生決策建議摘要 (使用 gemini-1.5-pro-latest)
    """
    if not ai_input:
        return "請先點擊「開始推薦」以產生餐廳清單。"

    if model is None:
        return "Gemini API 未設定。無法產生 AI 建議。"

    system_prompt = (
        "您是一位幽默且極具效率的午餐決策顧問。您的任務是根據提供的餐廳清單，"
        "分析其優缺點（如價格、評分、特色）並總結，幫助使用者在 3 家中快速做出決定。"
        "請以輕鬆口吻，用 **繁體中文** 撰寫一個約 **100 字** 的決策建議摘要，不超過 150 字。"
        "請避免使用 Markdown 標題和編號清單。"
    )

    user_query = f"這是三家推薦的餐廳清單：{ai_input}。請綜合考量其價格、評分和特色，為我提供一份快速決策建議。"

    try:
        response = await asyncio.to_thread(
            model.generate_content,
            contents=[user_query],
            generation_config=genai.GenerationConfig(
                temperature=0.7,
            ),
        )
        return response.text.strip()

    except Exception as e:
        error_msg = f"呼叫 Gemini API 失敗：{e}。"
        print(error_msg)
        return error_msg

In [69]:
async def serpapi_recommend_wrapper(location_text, keyword, price_filter, is_available):
    """
    Gradio 主要流程：
    1. 呼叫 SerpApi 抓資料
    2. 解析資料成 DataFrame
    3. 篩選 DataFrame
    4. 隨機推薦 3 家
    5. 呼叫 AI 摘要
    6. (新) 產生包含地址和地圖連結的 DataFrame
    7. (新) 回傳 DataFrame 和 AI 摘要
    """
    if SERPAPI_KEY is None:
        return pd.DataFrame(), "SerpApi 金鑰未設定。"

    params = {
      "engine": "google_maps",
      "q": keyword,
      "location": location_text,
      "google_domain": "google.com.tw",
      "hl": "zh-TW",
      "api_key": SERPAPI_KEY
    }
    if is_available:
        params["tbs"] = "oR:true"

    try:
        search = GoogleSearch(params)
        results = await asyncio.to_thread(search.get_dict)
        if "error" in results:
            return pd.DataFrame(), f"SerpApi 錯誤: {results['error']}"
    except Exception as e:
        return pd.DataFrame(), f"SerpApi API 呼叫失敗: {e}"

    df_dynamic_restaurants = parse_serpapi_to_df(results)
    if df_dynamic_restaurants.empty:
        return pd.DataFrame(), "在此範圍內找不到任何符合條件的餐廳。"

    price_list = price_filter if price_filter else df_dynamic_restaurants['價位'].unique()
    available_status = ['是'] if is_available else ['是', '否']
    filtered_df = df_dynamic_restaurants[
        df_dynamic_restaurants['價位'].isin(price_list) &
        df_dynamic_restaurants['是否營業'].isin(available_status)
    ].copy()
    if filtered_df.empty:
        return pd.DataFrame(), "所有找到的餐廳中，沒有符合「價位」和「營業」篩選的。"

    n_recommend = min(3, len(filtered_df))
    recommended_df = filtered_df.sample(n=n_recommend, replace=False)

    ai_input = "、".join([
        f"{row['餐廳名稱']}，價位{row['價位']}，評分{row['評分']}，特色：{row['特色描述']}"
        for _, row in recommended_df.iterrows()
    ])
    ai_summary = await generate_summary(ai_input)

    display_df = recommended_df.copy()

    display_df['Google Maps 連結'] = display_df['place_id'].apply(
        lambda pid: f"https://www.google.com/maps/search/?api=1&query_place_id={pid}"
    )

    display_df = display_df[[
        '餐廳名稱',
        '價位',
        '評分',
        '是否營業',
        '特色描述',
        'address',
        'Google Maps 連結'
    ]]

    return display_df, ai_summary

In [70]:
with gr.Blocks(title="動態餐廳決策器 (Gradio + SerpApi + Gemini)") as demo:
    gr.Markdown(
        """
        # 🍽️ 動態餐廳決策器 (Gradio + SerpApi + Gemini)
        ### 目標：輸入地點 -> 搜尋附近 -> AI 助您快速決策！
        **🚨 注意：請確保您已在 Colab Secrets 中設定 `gemini` 和 `SERPAPI_KEY`**
        """
    )

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## 1. 搜尋條件")

            location_input = gr.Textbox(
                label="搜尋地點 (必填)",
                value="國立台灣師範大學",
                info="輸入地址或地標"
            )
            keyword_input = gr.Textbox(
                label="餐廳關鍵字 (必填)",
                value="餐廳",
                placeholder="例如：義大利麵、素食、咖啡廳"
            )

            price_select = gr.CheckboxGroup(
                ['$', '$$', '$$$'],
                label="價位過濾 (可複選)",
                value=['$', '$$', '$$$'],
                info="($: 低, $$: 中, $$$: 高)"
            )
            available_toggle = gr.Checkbox(
                label="只看目前營業",
                value=True
            )

            recommend_btn = gr.Button("🚀 搜尋並推薦", variant="primary")

        with gr.Column(scale=3):
            gr.Markdown("## 2. 推薦清單 (隨機 3 家)")

            output_dataframe = gr.DataFrame(
                headers=["餐廳名稱", "價位", "評分", "營業", "特色描述", "地址", "Google Maps 連結"],
                datatype=["str", "str", "str", "str", "str", "str", "markdown"],
                label="隨機推薦結果"
            )

    gr.Markdown("## 3. 🤖 AI 決策建議")

    output_summary = gr.Textbox(
        label="AI 決策顧問報告",
        interactive=False,
        info="AI 將根據推薦結果，總結優缺點，幫助您快速做出選擇。",
        lines=5
    )

    recommend_btn.click(
        fn=serpapi_recommend_wrapper,
        inputs=[
            location_input,
            keyword_input,
            price_select,
            available_toggle
        ],
        outputs=[
            output_dataframe,
            output_summary
        ]
    )

In [71]:
demo.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4d981b1971454d38f5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://4d981b1971454d38f5.gradio.live
